# Creating the Node Class for the Decision Tree......

In [1]:
# here we are just creating the node for the tree.....where
 #feature-> the feature on which we splitted the tree...
 #value->the value of the continuous feature at wich we alitted
 #output->the output for the current node if we donot split further
 #gain_ratio-> gain_ratio for that node....
 #y will store the y_train of that node....which will later be used to calculate the entropy for that node.....
class Node:
    def __init__(self,feature=None,value=None,output=None,gain_ratio=None,level=None,y=None):
        self.feature=feature;
        self.value=value;
        self.output=output;
        self.gain_ratio=gain_ratio;
        self.level=level;
        self.y=y;
        self.right=None;
        self.left=None;
        

# Creating All the Functions which are requied for making the Decision Tree....

In [2]:
def find_split_info(d1,d2):
    d=d1+d2;
    num1 = d1/d
    num2 = d2/d
    return (-1)*((num1*math.log2(num1)/math.log2(2))+(num2*math.log2(num2)/math.log2(2)))
    

In [3]:
def find_entropy(y):
    dic = Counter(y)
    entropy = 0
    keys = dic.keys()
    total = len(y)
    for i in keys:
        key_value = dic[i]
        p = (key_value/total)
        entropy+=-1*(p*math.log2(p));
    return entropy;
    

In [4]:
def find_gain_ratio(x,y,i):
    list1=x[:,i].copy();
    list1.sort();
    value=0;
    parent_entropy=find_entropy(y);
    gain_ratio=-1;
    for j in range(0,len(list1)-1):
        avg=(list1[j]+list1[j+1])/2;
        #suppose if list1 is[2,2,3,4,4,5,6,] so we will get avg for 1st time as (2+2)/2=2 and we need to avoid that
        #so used this if condition...
        if(avg==list1[j]):
            continue;
            
        less_than_avg_y=y[x[:,i]<avg];
        more_than_avg_y=y[x[:,i]>=avg];
        d1=len(less_than_avg_y);
        d2=len(more_than_avg_y);
        
        split_info=find_split_info(d1,d2);
        
        
        e1=find_entropy(less_than_avg_y);
        e2=find_entropy(more_than_avg_y);
        
        
        info_req=e1*(d1/(d1+d2))+e2*(d2/(d1+d2));
        info_gain=parent_entropy-info_req;
        temp_gain_ratio=info_gain/split_info;
        
        if(temp_gain_ratio>=gain_ratio):
            gain_ratio=temp_gain_ratio;
            value=avg;
     # it returns the gain_ratio at which we are splitting and also the value of the feature at which we are splitting.
    # to the find_feature_to_split() function which has called this function..........
    return [gain_ratio,value];

### There are many features available for the splitting so it will find the feature which provide the best gain ratio and returns the feature_no,value pt of feature at which we split(because the datapoints are continuous),and the gain ratio at which we split Node

In [5]:
def find_feature_to_split(x,y):
    # feature_no has the value that we are taking th 0th ,1st or which feature
    feature_no=0;
    value=0;
    gain_ratio=-1;
    for i in range(0,x.shape[1]):
        list1=find_gain_ratio(x,y,i);
        temp_gain_ratio=list1[0];
        temp_value=list1[1];
        if(temp_gain_ratio>gain_ratio):
            gain_ratio=temp_gain_ratio;
            feature_no=i;
            value=temp_value;
     #It returns the feature_no,the value of feature for split and the gain_ratio for split.       
    return [feature_no,value,gain_ratio];
        
    

### This cell is used for the construction of the tree....if u need to make a tree then call Construct_tree function with the input and output of train_dataset.....

In [6]:
def construct_tree(x_train,y_train,level=0):
    # base case i.e. pure node
    if(len(set(y_train))==1):
        return Node(output=y_train[0],gain_ratio=0.0,level=level,y=y_train);
    
    list1=find_feature_to_split(x_train,y_train);

    
    # this list contains the feature_name,,,value of feature at which we are splitting,,,and the gain_ratio for the split
    node =Node(list1[0],list1[1],Counter(y_train).most_common()[0][0],list1[2],level,y_train);
    
    node.left=construct_tree(x_train[x_train[:,list1[0]]>=list1[1]],y_train[x_train[:,list1[0]]>=list1[1]],level+1);
    node.right=construct_tree(x_train[x_train[:,list1[0]]<list1[1]],y_train[x_train[:,list1[0]]<list1[1]],level+1);
    return node;
    
        
        
        
        

# Making Of Tree Is over.............

# It's Time to Print the tree
## And we are Using the Print Funtion to print the tree and this function takes the root of the tree and the feature_names of the dataset

In [7]:
def print_tree(root,feature):
    print("Level",root.level);
    temp=Counter(root.y).most_common();
    if(len(temp)==1):
        print("Count of",temp[0][0],"=",temp[0][1]);
        print("Current Entropy is =",find_entropy(root.y));
        print("Reached Leaf Node");
        print();
        print();
    else:
        for i in range(len(temp)-1,-1,-1):
            print("Count of",temp[i][0],"=",temp[i][1]);
        print("Current Entropy is =",find_entropy(root.y));
        print("Splitting on feature",feature[root.feature],"with gain ratio =",root.gain_ratio);
        print();
        print();
    if(root.left!=None):
        print_tree(root.right,feature);
        print_tree(root.left,feature);
        

# This is a function which predicts the output for a given input feature values

In [27]:
def predict_for_i(i,node):
    if(node.left==None and node.right==None):
        return node.output;
    elif(i[node.feature]<node.value):
        return predict_for_i(i,node.right);
    else:
        return predict_for_i(i,node.left);
        
        
        

# Creating And Printing The Tree for the OR(Gate)....

In [31]:
from sklearn import datasets;
import math;
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np;
from collections import Counter
iris = datasets.load_iris()
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state = 1);
#root=construct_tree(x_train,y_train,0);
root=construct_tree(np.array([[1,1],[0,1],[1,0],[0,0]]),np.array([1,1,1,0]));
print_tree(root,["x1","x2"]);

Level 0
Count of 0 = 1
Count of 1 = 3
Current Entropy is = 0.8112781244591328
Splitting on feature x1 with gain ratio = 0.31127812445913283


Level 1
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on feature x2 with gain ratio = 1.0


Level 2
Count of 0 = 1
Current Entropy is = 0.0
Reached Leaf Node


Level 2
Count of 1 = 1
Current Entropy is = 0.0
Reached Leaf Node


Level 1
Count of 1 = 2
Current Entropy is = 0.0
Reached Leaf Node




# This Cell Is Used To Make and Print Tree For The Iris Dataset.....

In [9]:
iris = datasets.load_iris()
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state = 1)
node1=construct_tree(x_train,y_train);
print_tree(node1,iris.feature_names);

Level 0
Count of 1 = 34
Count of 0 = 37
Count of 2 = 41
Current Entropy is = 1.5807197138422104
Splitting on feature petal length (cm) with gain ratio = 1.0000000000000002


Level 1
Count of 0 = 37
Current Entropy is = 0.0
Reached Leaf Node


Level 1
Count of 1 = 34
Count of 2 = 41
Current Entropy is = 0.993707106604508
Splitting on feature petal width (cm) with gain ratio = 0.6610420198933152


Level 2
Count of 2 = 4
Count of 1 = 33
Current Entropy is = 0.49418293484978865
Splitting on feature petal length (cm) with gain ratio = 0.6941833044972409


Level 3
Count of 1 = 32
Current Entropy is = 0.0
Reached Leaf Node


Level 3
Count of 1 = 1
Count of 2 = 4
Current Entropy is = 0.7219280948873623
Splitting on feature sepal length (cm) with gain ratio = 0.33155970728682876


Level 4
Count of 1 = 1
Count of 2 = 1
Current Entropy is = 1.0
Splitting on feature sepal width (cm) with gain ratio = 1.0


Level 5
Count of 2 = 1
Current Entropy is = 0.0
Reached Leaf Node


Level 5
Count of 1 = 1
C

# This is just for printing the accuracy of our Decision Tree on Iris Dataset and some other error measuring techniques such as confusion matrix and classifier report...

In [39]:
from sklearn.metrics import confusion_matrix;
from sklearn.metrics import classification_report;
y_pred=[];
for i in range(len(x_test)):
    y_pred.append(predict_for_i(x_test[i],node1));
print("Accuracy of prediction is->",accuracy_score(y_pred,y_test));
print();
print();
print("Confusion Matrix.....")
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print();
print();
print("Classifier report.....")
print(classification_report(y_pred=y_pred, y_true=y_test))

Accuracy of prediction is-> 0.9736842105263158


Confusion Matrix.....
[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


Classifier report.....
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



In [36]:
print(y_pred);
print(list(y_test));

[0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0]
[0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0]
